In [1]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols
from statsmodels.stats.anova import AnovaRM
# from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
# import data
data = pd.ExcelFile('/Users/jessebecklevisohn/Documents/GitHub/group3-project/Cleaned_data_Notcollinear?.xlsx')
data1 = pd.ExcelFile('/Users/jessebecklevisohn/Documents/GitHub/group3-project/Cleaned_data.xlsx')

In [3]:
#loading all sheets as seperate dataframe objects

mcav_control = pd.read_excel(data, sheet_name ='Mcav Control')

mcav_elevated_PCO2 = pd.read_excel(data, sheet_name ='Mcav Elevated PCO2')

ofav_control = pd.read_excel(data, sheet_name ='Ofav Control')

ofav_elevated_PCO2 = pd.read_excel(data, sheet_name = 'Ofav Elevated P CO2')

pcli_control = pd.read_excel(data, sheet_name = 'Pcli Control')

pcli_elevated_PCO2 = pd.read_excel(data, sheet_name = 'Pcli Elevated PCO2')

In [4]:
# Combines control and elevated pCO2 sheets into one pandas dataframe
mcav = pd.concat([mcav_control, mcav_elevated_PCO2], axis=0).reset_index(drop=True)
ofav = pd.concat([ofav_control, ofav_elevated_PCO2], axis=0).reset_index(drop=True)
pcli = pd.concat([pcli_control, pcli_elevated_PCO2], axis=0).reset_index(drop=True)

In [5]:
# Adding all the sheets to a list
species = {"mcav":mcav, "ofav":ofav, "pcli":pcli}
pcli.shape

(80, 9)

In [6]:
# Rename the columns of the sheet
for key in species:
    species[key] = species[key].rename(columns={"Percent change Surface Area Density": "SAD", 
                                  "Calcification rate (mgCaCO3/cm2/day)": "calcification_rate", 
                                  "Tissue growth (mm2/day)": "tissue_growth",
                                  "Exposure Period": "exposure_period"})

In [7]:
# Get rid of all null values in the columns that we care about
for key in species:
    species[key] = species[key].loc[
        (species[key]["calcification_rate"].notnull()) & 
        (species[key]["tissue_growth"].notnull()) & 
        (species[key]["SAD"].notnull()) &
        (species[key]["exposure_period"].notnull()),:]

In [31]:
dependent_variables = ["calcification_rate", "tissue_growth", "SAD"]
elevated_pCO2 = {"mcav":{}, "ofav": {}, "pcli": {}}
pCO2_by_exposure_period = {"mcav":{}, "ofav": {}, "pcli": {}}

# Iterate over each species and each dependant variable that we care about
for key in species:
    for var in dependent_variables:
        # Conduct a repeated measures ANOVA and save the result to a dictionary of dictionaries
        anova_rm = AnovaRM(data=species[key], depvar=var, subject='Sample', within=['Elevated', 'exposure_period']).fit()
        
        # Create a tuple with the p values for elevated, and elevated*exposure_period and put it in the two dictionaries
        elevated_pCO2[key][var] = anova_rm.anova_table["Pr > F"][0]
        pCO2_by_exposure_period[key][var] = anova_rm.anova_table["Pr > F"][2]



In [32]:
elevated_pCO2 = pd.DataFrame(elevated_pCO2)
elevated_pCO2 = elevated_pCO2.rename(columns={"mcav":"Montastraea cavernosa", "pcli":"Pseudodiploria clivosa", "ofav":"Orbicella faveolata"})
elevated_pCO2 = elevated_pCO2.rename(index={"calcification_rate":"Calcification rate (mgCaCO3/cm2/day)", "SAD":"Percent change Surface Area Density", "tissue_growth":"Tissue growth (mm2/day)"})
elevated_pCO2 = elevated_pCO2.style.set_caption("P values for Coral in Elevated vs. Control pCO2")
elevated_pCO2 = elevated_pCO2.background_gradient(axis=None, vmin=0.05, vmax=1, cmap="GnBu")
elevated_pCO2

,Montastraea cavernosa,Orbicella faveolata,Pseudodiploria clivosa
Calcification rate (mgCaCO3/cm2/day),0.877022,0.623588,0.810138
Tissue growth (mm2/day),0.225775,0.181145,0.449543
Percent change Surface Area Density,0.424498,0.125122,0.537035


In [33]:
pCO2_by_exposure_period = pd.DataFrame(pCO2_by_exposure_period)
pCO2_by_exposure_period = pCO2_by_exposure_period.rename(columns={"mcav":"Montastraea cavernosa", "pcli":"Pseudodiploria clivosa", "ofav":"Orbicella faveolata"})
pCO2_by_exposure_period = pCO2_by_exposure_period.rename(index={"calcification_rate":"Calcification rate (mgCaCO3/cm2/day)", "SAD":"Percent change Surface Area Density", "tissue_growth":"Tissue growth (mm2/day)"})
pCO2_by_exposure_period = pCO2_by_exposure_period.style.set_caption("P values for the Combined Effects of pCO2 Levels and Exposure Period")
pCO2_by_exposure_period = pCO2_by_exposure_period.background_gradient(axis=None, vmin=0.05, vmax=1, cmap="GnBu")
pCO2_by_exposure_period

,Montastraea cavernosa,Orbicella faveolata,Pseudodiploria clivosa
Calcification rate (mgCaCO3/cm2/day),0.904015,0.731945,0.086428
Tissue growth (mm2/day),0.454285,0.932652,0.810963
Percent change Surface Area Density,0.536866,0.749440,0.510909


In [ ]:
# Tukey test for if any of the AnovaRM tests show a relevant correlation

# dependent_variables = ["calcification_rate", "tissue_growth", "SAD"]
# tukeys = {"mcav":{}, "ofav": {}, "pcli": {}}

# # Iterate over each species and each dependant variable that we care about
# for key in species:
#     for var in dependent_variables:
#         tukey = pairwise_tukeyhsd(endog=species[key][var], groups=species[key]["Elevated"], alpha=0.05) # Conduct a Tukey Test across the pCO2 levels 
#         tukeys[key][var] = tukey # Save the result of the tukey test to a dictionary of dictionaries

# print(tukeys["mcav"]["calcification_rate"])